In [6]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from scipy.optimize import minimize_scalar
from datetime import datetime, date
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas import DataFrame
import pickle

## Black-Scholes Model
Implementation of the Black-Scholes Model for the price calculation of european Call/Put options.
<br>
<br>
__Symbols:__
<br>
S = Current Stock Price
<br>
K = Strike Price
<br>
t = time to maturity
<br>
r = risk-free rate (US Treasury Bonds, because of missing API support)
<br>
sigma = volatility

In [7]:
# calculation
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

In [8]:
# european call
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

# european put
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

## Getting Data via Alpha Vantage

In [9]:
today = date.today()
one_year_ago = today.replace(year=today.year-1)

### getting company data

In [11]:
# just 4 companys due to API restrictions
dax_40 = pickle.load(open("stock_timeseries_dict.p", "rb"))
dax_40["ADS.DE"]

,open,high,low,close,volume
2021-05-31,299.00,302.90,297.65,298.35,279491
2021-06-01,300.80,301.00,296.65,298.00,456486
2021-06-02,298.95,299.15,295.90,298.50,290081
2021-06-03,298.55,299.85,295.35,298.95,276563
2021-06-04,298.00,299.25,295.55,296.60,325720
...,...,...,...,...,...
2022-05-23,176.42,177.10,172.44,175.48,563180
2022-05-24,172.72,175.28,170.84,171.20,671631
2022-05-25,173.54,173.54,165.80,172.08,825517
2022-05-26,173.30,177.20,173.00,176.76,578826


In [12]:
for k in dax_40.keys():
    dax_40[k] = dax_40[k].sort_index(ascending = False)
    dax_40[k] = dax_40[k].dropna()
    dax_40[k] = dax_40[k].assign(close_day_before=dax_40[k].close.shift(-1))
    dax_40[k]['returns'] = ((dax_40[k].close - dax_40[k].close_day_before)/dax_40[k].close_day_before)
dax_40["ADS.DE"]

,open,high,low,close,volume,close_day_before,returns
2022-05-27,178.30,180.94,175.56,179.96,597597,176.76,0.018104
2022-05-26,173.30,177.20,173.00,176.76,578826,172.08,0.027197
2022-05-25,173.54,173.54,165.80,172.08,825517,171.20,0.005140
2022-05-24,172.72,175.28,170.84,171.20,671631,175.48,-0.024390
2022-05-23,176.42,177.10,172.44,175.48,563180,172.80,0.015509
...,...,...,...,...,...,...,...
2021-06-04,298.00,299.25,295.55,296.60,325720,298.95,-0.007861
2021-06-03,298.55,299.85,295.35,298.95,276563,298.50,0.001508
2021-06-02,298.95,299.15,295.90,298.50,290081,298.00,0.001678
2021-06-01,300.80,301.00,296.65,298.00,456486,298.35,-0.001173


## Calculating Option Price

### setting up parameters for Calculation

In [13]:
df_risk_free = pickle.load(open("risk_free_rate.p", "rb"))
df_risk_free = df_risk_free[["date", "value"]].copy()
df_risk_free = df_risk_free.sort_index(ascending = False)
df_risk_free
#uty = (web.DataReader("^TNX", 'yahoo', today.replace(day=today.day-3), today)['Close'].iloc[-1])/100

,date,value
27,2022-04-01,0.007500
26,2022-03-01,0.002800
25,2022-02-01,0.001500
24,2022-01-01,-0.001148
23,2021-12-01,-0.003843
22,2021-11-01,-0.003136
21,2021-10-01,-0.002043
20,2021-09-01,-0.003627
19,2021-08-01,-0.005386
18,2021-07-01,-0.004514


In [14]:
stock = 'BAS.DE' # Company ticker
expiry = '2022-12-14'
strike_price = 59
relation = 0.1 #Bezugsverhältnis

In [20]:
# just execute, no changes
def calc_option_price(df, df_risk_free, strike_price, t1, t2, relation = 0.1, option_type = "call"):

    sigma = np.sqrt(252) * df['returns'].std()
    lcp = df['close'].iloc[1]
    t = (datetime.strptime(t2, "%Y-%m-%d") - datetime.strptime(t1, "%Y-%m-%d")).days / 365
    uty = df_risk_free["value"].iloc[1]


    #print("+++ " + str(stock) + " +++")

    if (option_type == "put"):
        return bs_put(lcp, strike_price, t, uty, sigma)*relation

    elif (option_type == "call"):
        return bs_call(lcp, strike_price, t, uty, sigma)*relation
    else:
        raise Exception("option_type must be either call or put.")

In [21]:
a = calc_option_price(dax_40[stock], df_risk_free, 60, "2022-05-30", expiry)
print(a)

0.14457309886913147


### Alles hier nach noch nicht fertig

## Implied volatility

In [11]:
def implied_vol(opt_value, S,K,T,r,type_='call'):
    
    def call_obj(sigma):
        return abs(bs_call(S,K,T,r,sigma) - opt_value)
    
    def put_obj(sigma):
        return abs(bs_put(S,K,T,r,sigma) - opt_value)
    
    if type_ == 'call':
        res = minimize_scalar(call_obj, bounds=(0.01,6), method='bounded')
        return res.x
    elif type_ == 'put':
        res = minimize_scalar(put_obj, bounds=(0.01,6),
                              method='bounded')
        return res.x
    else:
        raise ValueError("type_ must be 'put' or 'call'")

In [ ]:
def vega(S, K, T, r, q = 0.0, sigma):
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    #q: continuous dividend rate
    
    D1 = d1(S,K,T,r,sigma)
    
    if (q != 0):
        # Vega for Dividend Paying Assets
        return 1 / np.sqrt(2 * np.pi) * S * np.exp(-q * T) * np.exp(-(D1 ** 2) * 0.5) * np.sqrt(T)
    else:
        # Vega for non-Dividend Paying Assets
        return S * norm.cdf(D1, 0.0, 1.0) * np.sqrt(T)

In [14]:
C = bs_call(lcp, strike_price, t, uty, sigma)
iv = implied_vol(C,lcp,strike_price,t,uty)
print(iv)
print(sigma)

0.28361219679799693
0.28361285363237027
